## Utility functions

In [6]:
def parse_input(filename, line_fn=None):
    with open(filename) as f:
        content = f.readlines()
        content = [x.strip() for x in content]
        if (line_fn):
            content = list(map(lambda line: line_fn(line), content))
        return content
    
def split_integers(line):
    return list(map(lambda el: int(el), line.split()))

## Part 1 A

The captcha requires you to review a sequence of digits (your puzzle input) and find the sum of all digits that match the next digit in the list. The list is circular, so the digit after the last digit is the first digit in the list.

For example:

    1122 produces a sum of 3 (1 + 2) because the first digit (1) matches the second digit and the third digit (2) matches the fourth digit.
    1111 produces 4 because each digit (all 1) matches the next.
    1234 produces 0 because no digit matches the next.
    91212129 produces 9 because the only digit that matches the next one is the last digit, 9.


In [20]:
def day1_captcha(captcha):
    sum = 0
    for i in range(len(captcha)):
        this_char = captcha[i]
        last_char = captcha[i-1]
        if this_char == last_char:
            sum += int(this_char)
    return sum

assert day1_captcha("1122") == 3, day1_captcha("1122")
assert day1_captcha("1111") == 4, day1_captcha("1111")
assert day1_captcha("1234") == 0, day1_captcha("1234")
assert day1_captcha("91212129") == 9, day1_captcha("91212129")

day1_captcha( parse_input('input_day01a')[0] )

1144

## Part 1 B

Now, instead of considering the next digit, it wants you to consider the digit halfway around the circular list. That is, if your list contains 10 items, only include a digit in your sum if the digit 10/2 = 5 steps forward matches it. Fortunately, your list has an even number of elements.

For example:

    1212 produces 6: the list contains 4 items, and all four digits match the digit 2 items ahead.
    1221 produces 0, because every comparison is between a 1 and a 2.
    123425 produces 4, because both 2s match each other, but no other digit has a match.
    123123 produces 12.
    12131415 produces 4.


In [15]:
def day1_captcha(captcha):
    sum = 0
    for i in range(len(captcha)):
        this_char = captcha[i]
        last_char = captcha[i- int(len(captcha)/2) ]
        if this_char == last_char:
            sum += int(this_char)
    return sum

assert day1_captcha("1212") == 6, day1_captcha("1212")
assert day1_captcha("1221") == 0, day1_captcha("1221")
assert day1_captcha("123425") == 4, day1_captcha("123425")
assert day1_captcha("123123") == 12, day1_captcha("123123")
assert day1_captcha("12131415") == 4, day1_captcha("12131415")

day1_captcha( parse_input('input_day01a')[0] )

1194

## Part 2A

For each row, determine the difference between the largest value and the smallest value; the checksum is the sum of all of these differences.

For example, given the following spreadsheet:

5 1 9 5
7 5 3
2 4 6 8

    The first row's largest and smallest values are 9 and 1, and their difference is 8.
    The second row's largest and smallest values are 7 and 3, and their difference is 4.
    The third row's difference is 6.

In this example, the spreadsheet's checksum would be 8 + 4 + 6 = 18.

In [45]:
def day2_checksum(sheet):
    return sum(list(map(lambda line: max(line)-min(line), sheet)))

test_checksum = day2_checksum([[5,1,9,5],[7,5,3],[2,4,6,8]])
assert test_checksum == 18, test_checksum

day2_checksum( parse_input('input_day02a', lambda line: split_integers(line)) )


41919

## Part 2B

It sounds like the goal is to find the only two numbers in each row where one evenly divides the other - that is, where the result of the division operation is a whole number. They would like you to find those numbers on each line, divide them, and add up each line's result.

For example, given the following spreadsheet:

5 9 2 8
9 4 7 3
3 8 6 5

    In the first row, the only two numbers that evenly divide are 8 and 2; the result of this division is 4.
    In the second row, the two numbers are 9 and 3; the result is 3.
    In the third row, the result is 2.

In this example, the sum of the results would be 4 + 3 + 2 = 9.

In [68]:
def day2_checksum(sheet):
    def calc_line(line):
        line = sorted(line)
        for i in range(len(line)):
            for j in range(i):
                if line[i] % line[j] == 0:
                    return int(line[i]/line[j])
    return sum(list(map(lambda line: calc_line(line), sheet)))

test_checksum = day2_checksum([[5,9,2,8],[9,4,7,3],[3,8,6,5]])
assert test_checksum == 9, test_checksum

day2_checksum( parse_input('input_day02a', lambda line: split_integers(line)) )

303

## Part 3A

You come across an experimental new kind of memory stored on an infinite two-dimensional grid.

Each square on the grid is allocated in a spiral pattern starting at a location marked 1 and then counting up while spiraling outward. For example, the first few squares are allocated like this:

17  16  15  14  13
18   5   4   3  12
19   6   1   2  11
20   7   8   9  10
21  22  23---> ...

While this is very space-efficient (no squares are skipped), requested data must be carried back to square 1 (the location of the only access port for this memory system) by programs that can only move up, down, left, or right. They always take the shortest path: the Manhattan Distance between the location of the data and square 1.

For example:

    Data from square 1 is carried 0 steps, since it's at the access port.
    Data from square 12 is carried 3 steps, such as: down, left, left.
    Data from square 23 is carried only 2 steps: up twice.
    Data from square 1024 must be carried 31 steps.

How many steps are required to carry the data from the square identified in your puzzle input all the way to the access port?

Your puzzle input is 277678.

In [5]:
import math

# OEIS A214526
def get_x_y(n):
  sr = int(math.sqrt(n-1))
  sr = sr-1+(sr&1)
  rm = n-sr*sr
  d = (sr+1)/2
  if rm<=sr+1:
    return -d+rm,d
  if rm<=sr*2+2:
    return d,d-(rm-(sr+1))
  if rm<=sr*3+3:
    return d-(rm-(sr*2+2)),-d
  return -d,-d+rm-(sr*3+3) 

def day3_distance(n):
  xy = get_x_y(n)
  return abs(xy[0]) + abs(xy[1])

assert day3_distance(1) == 0, day3_distance(1)
assert day3_distance(12) == 3, day3_distance(12)
assert day3_distance(23) == 2, day3_distance(23)
assert day3_distance(1024) == 31, day3_distance(1024)

day3_distance(277678)

475.0

## Part 3B

As a stress test on the system, the programs here clear the grid and then store the value 1 in square 1. Then, in the same allocation order as shown above, they store the sum of the values in all adjacent squares, including diagonals.

So, the first few squares' values are chosen as follows:

    Square 1 starts with the value 1.
    Square 2 has only one adjacent filled square (with value 1), so it also stores 1.
    Square 3 has both of the above squares as neighbors and stores the sum of their values, 2.
    Square 4 has all three of the aforementioned squares as neighbors and stores the sum of their values, 4.
    Square 5 only has the first and fourth squares as neighbors, so it gets the value 5.

Once a square is written, its value does not change. Therefore, the first few squares would receive the following values:

147  142  133  122   59
304    5    4    2   57
330   10    1    1   54
351   11   23   25   26
362  747  806--->   ...

What is the first value written that is larger than your puzzle input?

Your puzzle input is still 277678.

In [7]:
# Meh, boring.  This is just https://oeis.org/A141481/b141481.txt and I'm not rewriting it.

## Part 4A

A passphrase consists of a series of words (lowercase letters) separated by spaces.  To ensure security, a valid passphrase must contain no duplicate words.

For example:

    aa bb cc dd ee is valid.
    aa bb cc dd aa is not valid - the word aa appears more than once.
    aa bb cc dd aaa is valid - aa and aaa count as different words.

The system's full passphrase list is available as your puzzle input. How many passphrases are valid?

In [15]:
def day4_passphrase_is_valid(passphrase):
    return len(set(passphrase)) == len(passphrase)

assert day4_passphrase_is_valid(['aa','bb','cc','dd','ee'])
assert not day4_passphrase_is_valid(['aa','bb','cc','dd','aa'])
assert day4_passphrase_is_valid(['aa','bb','cc','dd','aaa'])

valid_passphrases = list(filter(lambda p: day4_passphrase_is_valid(p), parse_input('input_day04a', lambda line: line.split()) ))
len(valid_passphrases)

466

## Part 4B

Now, a valid passphrase must contain no two words that are anagrams of each other - that is, a passphrase is invalid if any word's letters can be rearranged to form any other word in the passphrase.

For example:

    abcde fghij is a valid passphrase.
    abcde xyz ecdab is not valid - the letters from the third word can be rearranged to form the first word.
    a ab abc abd abf abj is a valid passphrase, because all letters need to be used when forming another word.
    iiii oiii ooii oooi oooo is valid.
    oiii ioii iioi iiio is not valid - any of these words can be rearranged to form any other word.

Under this new system policy, how many passphrases are valid?

In [18]:
def day4_passphrase_is_valid(passphrase):
    passphrase = list(map(lambda word: ''.join(sorted(word)), passphrase))
    return len(set(passphrase)) == len(passphrase)

assert day4_passphrase_is_valid(['abcde','fghij'])
assert not day4_passphrase_is_valid(['abcde','xyz','ecdab'])

valid_passphrases = list(filter(lambda p: day4_passphrase_is_valid(p), parse_input('input_day04a', lambda line: line.split()) ))
len(valid_passphrases)

251

## Part 5A

The message includes a list of the offsets for each jump. Jumps are relative: -1 moves to the previous instruction, and 2 skips the next one. Start at the first instruction in the list. The goal is to follow the jumps until one leads outside the list.

In addition, these instructions are a little strange; after each jump, the offset of that instruction increases by 1. So, if you come across an offset of 3, you would move three instructions forward, but change it to a 4 for the next time it is encountered.

For example, consider the following list of jump offsets:

0
3
0
1
-3

Positive jumps ("forward") move downward; negative jumps move upward. For legibility in this example, these offset values will be written all on one line, with the current instruction marked in parentheses. The following steps would be taken before an exit is found:

    (0) 3  0  1  -3  - before we have taken any steps.
    (1) 3  0  1  -3  - jump with offset 0 (that is, don't jump at all). Fortunately, the instruction is then incremented to 1.
     2 (3) 0  1  -3  - step forward because of the instruction we just modified. The first instruction is incremented again, now to 2.
     2  4  0  1 (-3) - jump all the way to the end; leave a 4 behind.
     2 (4) 0  1  -2  - go back to where we just were; increment -3 to -2.
     2  5  0  1  -2  - jump 4 steps forward, escaping the maze.

In this example, the exit is reached in 5 steps.

In [8]:
def day5_findexit(instrs):
    posn = 0
    time = 0
    while posn in range(len(instrs)):
        time += 1
        cur_jump = instrs[posn]
        instrs[posn] += 1
        posn += cur_jump
    return time

assert day5_findexit([0,3,0,1,-3])==5, day5_findexit([0,3,0,1,-3])

day5_findexit(parse_input('input_day05a', lambda line: int(line)))

373160

## Part 5B

Now, the jumps are even stranger: after each jump, if the offset was three or more, instead decrease it by 1. Otherwise, increase it by 1 as before.

Using this rule with the above example, the process now takes 10 steps, and the offset values after finding the exit are left as 2 3 2 3 -1.

How many steps does it now take to reach the exit?

In [9]:
def day5_findexit(instrs):
    posn = 0
    time = 0
    while posn in range(len(instrs)):
        time += 1
        cur_jump = instrs[posn]
        if cur_jump >= 3:
            instrs[posn] -= 1
        else:
            instrs[posn] += 1
        posn += cur_jump
    return time

assert day5_findexit([0,3,0,1,-3])==10, day5_findexit([0,3,0,1,-3])

day5_findexit(parse_input('input_day05a', lambda line: int(line)))

26395586